In [107]:
import requests
import datetime
from urllib.parse import urlencode

In [108]:
import base64

In [109]:
client_id= "49e92e9173284e27b12af17f66c1f3e6"
client_secret= "725fc8b1ecdf4d34b5f9598d713a975e"

In [110]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        } 
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            raise Exception("Could not authenticate client.")
            # return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
  
    
    def get_episodes(self, _id):
        base_url= "https://api.spotify.com/v1"
        end_point=f"{base_url}/v1/episodes"
    
    def search(self, query, search_type='artist' ): # type
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        endpoint = "https://api.spotify.com/v1/search"
        data = urlencode({"q": query, "type": search_type.lower()})
        lookup_url = f"{endpoint}?{data}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):  
            return {}
        return r.json()

In [111]:
spotify = SpotifyAPI(client_id, client_secret)

In [112]:
spotify.search("joe rogan", search_type="artist")

{'artists': {'href': 'https://api.spotify.com/v1/search?query=joe+rogan&type=artist&offset=0&limit=20',
  'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6lrt7LngdzxaQtLIXMraSR'},
    'followers': {'href': None, 'total': 206132},
    'genres': ['comedy', 'comic'],
    'href': 'https://api.spotify.com/v1/artists/6lrt7LngdzxaQtLIXMraSR',
    'id': '6lrt7LngdzxaQtLIXMraSR',
    'images': [{'height': 1029,
      'url': 'https://i.scdn.co/image/9885ca67dbad4bad32928dac938a806e3c821b6f',
      'width': 1000},
     {'height': 658,
      'url': 'https://i.scdn.co/image/2bfc26d33dfea413b462546de85fba3ad5b6405a',
      'width': 640},
     {'height': 206,
      'url': 'https://i.scdn.co/image/c2bede582de646e64890eca2c5267b0ba2890e31',
      'width': 200},
     {'height': 66,
      'url': 'https://i.scdn.co/image/83aaf7e95d9d00160e851f9fc41c182c88787cd9',
      'width': 64}],
    'name': 'Joe Rogan',
    'popularity': 53,
    'type': 'artist',
    'uri': 'spotify:artist:6l

In [113]:
spotify.get_episodes("6lrt7LngdzxaQtLIXMraSR")
